In [1]:
# coding: utf-8
# a simple MLP network
import numpy as np
import scipy.io as spio
import tensorflow as tf

hidden_size=10
output_size=3
batch_size=50
beta = 0.1 # regularization

sess = tf.Session()

In [2]:
def load_data():
    data_dir = "train_test"
    X_train = spio.loadmat(data_dir + "/" + "train_data.mat")["train_data"]
    X_test = spio.loadmat(data_dir + "/" + "test_data.mat")["test_data"]
    y_train = spio.loadmat(data_dir + "/" + "train_label.mat")["train_label"]
    y_test = spio.loadmat(data_dir + "/" + "test_label.mat")["test_label"]
    return X_train, X_test, y_train, y_test

In [3]:
def normalize(X):
    mean = X.mean(axis=0)
    var = X.std(axis=0)
    return (X - mean) / var, mean, var

In [4]:
def lb2vec(y_in):
    y = y_in.ravel()
    a = y - y.min()
    b = np.zeros((a.size, a.max()+1))
    b[np.arange(a.size),a] = 1
    return b

In [5]:
X = normalize(X_train)
X = np.asarray([[1,2,3],[0.1, 0.4, 0.8]])
mean = X.mean(axis=0)
var = X.var(axis=0)
y = X - mean
print mean
print y / var
print var

NameError: name 'X_train' is not defined

In [6]:
X_train, X_test, y_train, y_test = load_data()

X_train, X_train_mean, X_train_var = normalize(X_train)
# X_test = normalize(X_test)
X_test = (X_test - X_train_mean) / X_train_var
# map y labels to one-hot format
y_train = lb2vec(y_train)
y_test = lb2vec(y_test)

In [7]:
# MLP model
def mlp_model(input_layer, hidden_size=hidden_size):
    W1 = tf.Variable(tf.random_normal([310, hidden_size], mean=0.0, stddev=1.0))
    b1 = tf.Variable(tf.random_normal([1, hidden_size], mean=0.0, stddev=1.0))
    hidden_layer = tf.matmul(input_layer, W1) + b1
    W2 = tf.Variable(tf.random_normal([hidden_size, output_size], mean=0.0, stddev=1.0))
    b2 = tf.Variable(tf.random_normal([1, output_size], mean=0.0, stddev=1.0))
    output_layer = tf.matmul(hidden_layer, W2) + b2
    
    regularizer = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(b2)
    return output_layer, regularizer

# pipeline
input_layer = tf.placeholder(tf.float32, shape=(None, 310))
y_label = tf.placeholder(tf.float32, shape=(None, output_size))
output_layer, regularizer = mlp_model(input_layer)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_layer, labels=y_label)) + beta * regularizer

# opt = tf.train.AdadeltaOptimizer(0.1)
opt = tf.train.GradientDescentOptimizer(0.05)
train_step = opt.minimize(loss)

In [8]:
# Accuracy
y_predict = tf.nn.softmax(output_layer)
correct_prediction = tf.equal(tf.argmax(y_predict, axis=1), tf.argmax(y_label, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [9]:
# sess.run(tf.nn.softmax_cross_entropy_with_logits(logits=output_layer, labels=y_label), feed_dict={input_layer: X_train, y_label: y_train})

In [10]:
# sess.run(b1, feed_dict={input_layer: X_test, y_label: y_test})

In [11]:
# sess.run(loss, feed_dict={input_layer: X_test, y_label: y_test})

In [12]:
# Train, mini-batch processing
def train():
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(1000):
        rand_index = np.random.choice(len(X_train), size=batch_size)
        rand_x = X_train[rand_index]
        rand_y = y_train[rand_index]
        train_dict = {input_layer: rand_x, y_label: rand_y}
        test_dict = {input_layer: X_test, y_label: y_test}
        sess.run(train_step, feed_dict=train_dict)
        if (i+1)% 25 == 0:
            print('Step #' + str(i+1))
            print('Train loss = ' + str(sess.run(loss, feed_dict=train_dict)))
            print('Test loss = ' + str(sess.run(loss, feed_dict=test_dict)))
            print('Accuracy(Test) = ' + str(sess.run(accuracy, feed_dict=test_dict)))
            
    return float(sess.run(accuracy, feed_dict=test_dict))
        
    

In [13]:
loss_list = []
for iters in range(10):
    print iters
    loss_list.append(train())

0
Step #25
Train loss = 127.35
Test loss = 130.114
Accuracy(Test) = 0.661808
Step #50
Train loss = 99.1208
Test loss = 100.994
Accuracy(Test) = 0.661808
Step #75
Train loss = 77.1488
Test loss = 78.4619
Accuracy(Test) = 0.661808
Step #100
Train loss = 60.0512
Test loss = 60.9116
Accuracy(Test) = 0.784257
Step #125
Train loss = 46.7437
Test loss = 47.2324
Accuracy(Test) = 0.830904
Step #150
Train loss = 36.3889
Test loss = 36.7486
Accuracy(Test) = 0.661808
Step #175
Train loss = 28.33
Test loss = 28.744
Accuracy(Test) = 0.830904
Step #200
Train loss = 22.0619
Test loss = 22.5507
Accuracy(Test) = 0.830904
Step #225
Train loss = 17.1898
Test loss = 17.7495
Accuracy(Test) = 0.830904
Step #250
Train loss = 13.3995
Test loss = 14.0185
Accuracy(Test) = 0.830904
Step #275
Train loss = 10.4535
Test loss = 11.0767
Accuracy(Test) = 0.830904
Step #300
Train loss = 8.15987
Test loss = 8.80021
Accuracy(Test) = 0.830904
Step #325
Train loss = 6.37987
Test loss = 7.03372
Accuracy(Test) = 0.830904
Step

In [ ]:
print "Average accuracy on test set(hidden_size=30): ", sum(loss_list) / len(loss_list)

In [14]:
print "Average accuracy on test set(hidden_size=10): ", sum(loss_list) / len(loss_list)

Average accuracy on test set(hidden_size=10):  0.830903768539


In [ ]:
print "Average accuracy on test set(hidden_size=3): ", sum(loss_list) / len(loss_list)